In [1]:
import os, math, json, random, time
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.amp import GradScaler, autocast
from PIL import Image
import pandas as pd
import numpy as np
import torchvision.transforms as T
import torchvision.models as models

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available(): torch.cuda.manual_seed_all(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

device: cuda


In [ ]:
# --- 1) Config
CSV_TRAIN = "index_train.csv"
CSV_VAL   = "index_val.csv"
CSV_TEST  = "index_test.csv"

T_LEN = 16            # window length (Number of f0 ... f15 in CSV)
IMG_SIZE = 224        # Resolution of frame (e.g. 224 -> 224 * 224)
BATCH_TRAIN = 8
BATCH_VAL   = 8
NUM_WORKERS = 4
PIN_MEMORY  = torch.cuda.is_available()

# Normalization ImageNet
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD  = (0.229, 0.224, 0.225)


In [ ]:
# --- 2) Dataset
class MultiTargetWindowDataset(Dataset):
    """
    One row of index_*.csv = 16 of frame path + 4 regression target(y_ttc_inv, y_dmin_inv, y_tstar_norm, y_dist_inv)
                      + 4 raw + 4 mask(0/1)

    Return:
      x: [T, 3, H, W] FloatTensor
      y: [4] FloatTensor   (Sequence: y_ttc_inv, y_dmin_inv, y_tstar_norm, y_dist_inv)
      m: [4] FloatTensor   (Valid mask of each target as 0/1)
    """
    def __init__(self, csv_path, t_len=16, image_size=224, augment=False):
        self.df = pd.read_csv(csv_path)
        self.t_len = t_len
        self.augment = augment

        # Pre-cache
        self.frame_cols = [f"f{k}" for k in range(self.t_len)]

        base_trans = [
            T.Resize((image_size, image_size)),
            T.ToTensor(),
            T.Normalize(IMAGENET_MEAN, IMAGENET_STD),
        ]
        if augment:
            self.transform = T.Compose([
                T.Resize(int(image_size*1.1)),
                T.RandomResizedCrop(image_size, scale=(0.75, 1.0)),
                T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
                T.RandomHorizontalFlip(),
                T.ToTensor(),
                T.Normalize(IMAGENET_MEAN, IMAGENET_STD),
            ])
        else:
            self.transform = T.Compose(base_trans)

    def __len__(self):
        return len(self.df)

    def _safe_load_rgb(self, path):
        '''
        디버깅용 이미지 예외처리
        '''
        try:
            with Image.open(path) as im:
                return im.convert("RGB")
        except Exception:
            return Image.fromarray(np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8))

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # Load frame stack
        imgs = []
        for col in self.frame_cols:
            p = row[col]
            img = self._safe_load_rgb(p)
            img = self.transform(img)
            imgs.append(img)
        x = torch.stack(imgs, dim=0)   # [T, 3, H, W]

        # Regression target
        y = torch.tensor([
            float(row["y_ttc_inv"]),
            float(row["y_dmin_inv"]),
            float(row["y_tstar_norm"]),
            float(row["y_dist_inv"]),
        ], dtype=torch.float32)

        # Mask(0/1, for hubor loss)
        m = torch.tensor([
            float(row["mask_ttc"]),
            float(row["mask_dmin"]),
            float(row["mask_tstar"]),
            float(row["mask_dist"]),
        ], dtype=torch.float32)

        return x, y, m


In [4]:
train_ds = MultiTargetWindowDataset(CSV_TRAIN, t_len=T_LEN, image_size=IMG_SIZE, augment=True)
val_ds   = MultiTargetWindowDataset(CSV_VAL,   t_len=T_LEN, image_size=IMG_SIZE, augment=False)
test_ds   = MultiTargetWindowDataset(CSV_TEST,   t_len=T_LEN, image_size=IMG_SIZE, augment=False)

In [5]:
train_dl = DataLoader(
    train_ds,
    batch_size=BATCH_TRAIN,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
    drop_last=True,
)

val_dl = DataLoader(
    val_ds,
    batch_size=BATCH_VAL,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
    drop_last=False,
)

test_dl = DataLoader(
    test_ds,
    batch_size=BATCH_VAL,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
    drop_last=False,
)

In [6]:
xb, yb, mb = next(iter(train_dl))
print("x:", xb.shape, "y:", yb.shape, "mask:", mb.shape)

x: torch.Size([8, 16, 3, 224, 224]) y: torch.Size([8, 4]) mask: torch.Size([8, 4])


In [ ]:
class CausalTCN(nn.Module):
    def __init__(self, c, ks=3, dilations=(1, 2, 4, 8), dropout=0.1):
        super().__init__()
        layers = []
        for d in dilations:
            pad = (ks - 1) * d  # causal padding
            layers += [
                nn.Conv1d(c, c, ks, padding=pad, dilation=d),
                nn.ReLU(),
                nn.Dropout(dropout),
            ]
        self.net = nn.Sequential(*layers)

    def forward(self, x):  # x:[B,C,T]
        y = self.net(x)
        return y[..., -1]  # Return last step [B,C]

class ResNetTemporalMultiHead(nn.Module):
    def __init__(self, T=16, feat_dim=512, head_dim=256, temporal="avg"):
        super().__init__()
        base = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        self.backbone = nn.Sequential(*list(base.children())[:-1])   # [B,512,1,1]
        self.temporal = temporal
        self.T = T

        # 실험할 때 빠르게 전환 위해 조건문으로 구성
        if temporal == "avg":
            self.temporal_pool = nn.Identity()   # 나중에 mean으로 처리
            temporal_out = feat_dim
        elif temporal == "tcn":
            self.temporal_pool = CausalTCN(feat_dim)
            temporal_out = feat_dim
        elif temporal == "gru":
            self.gru = nn.GRU(input_size=feat_dim, hidden_size=feat_dim, batch_first=True, bidirectional=False)
            temporal_out = feat_dim
        else:
            raise ValueError("temporal must be one of ['avg','tcn','gru']")

        def head():
            return nn.Sequential(nn.Linear(temporal_out, head_dim), nn.ReLU(), nn.Linear(head_dim, 1))

        self.head_ttc   = head()
        self.head_dmin  = head()
        self.head_tstar = head()
        self.head_dist  = head()

        '''
        Explain:
            Uncertainty parameter(log-sigma)
            This learnable weights automatically learns
            weight of multi-task weighted sum
        '''
        self.log_sigma = nn.ParameterDict({
            "ttc":   nn.Parameter(torch.tensor(0.0)),
            "dmin":  nn.Parameter(torch.tensor(0.0)),
            "tstar": nn.Parameter(torch.tensor(0.0)),
            "dist":  nn.Parameter(torch.tensor(0.0)),
        })

    def forward(self, x):  # x:[B,T,3,H,W]
        B, T = x.shape[:2]
        x = x.view(B*T, 3, x.size(3), x.size(4))
        f = self.backbone(x).flatten(1)          # [B*T,512]
        f = f.view(B, T, -1)                      # [B,T,512]

        if self.temporal == "avg":
            g = f.mean(dim=1)                     # [B,512]
        elif self.temporal == "tcn":
            g = f.transpose(1,2)                  # [B,512,T]
            g = self.temporal_pool(g).squeeze(-1) # [B,512]
        elif self.temporal == "gru":
            _, h = self.gru(f)                    # h:[1,B,512]
            g = h[-1]                             # [B,512]

        out = {
            "ttc":   self.head_ttc(g).squeeze(1),
            "dmin":  self.head_dmin(g).squeeze(1),
            "tstar": self.head_tstar(g).squeeze(1),
            "dist":  self.head_dist(g).squeeze(1),
        }
        return out

In [8]:
model = ResNetTemporalMultiHead(T=T_LEN, temporal="tcn").to(device)

print(model)

ResNetTemporalMultiHead(
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_ru

In [ ]:
# --- Util of unfreezing backbone ---
def freeze_backbone(m, freeze=True):
    for p in m.backbone.parameters():
        p.requires_grad = (not freeze)

In [ ]:
# Freeze for first 5 epoch → Only temporal/head learns
FREEZE_EPOCHS = 5
freeze_backbone(model, freeze=True)

In [ ]:
# Freeze BatchNorm
def freeze_bn(m):
    import torch.nn as nn
    if isinstance(m, (nn.BatchNorm2d, nn.SyncBatchNorm)):
        m.eval()
        for p in m.parameters():
            p.requires_grad_(False)

In [ ]:
# torchvision-specific unfreeze util
def unfreeze_backbone_stage_by_index(backbone: nn.Sequential, stage_idx: int):
    """
    Pop ResNet18 out from torchvision then wrap by nn.Sequential:
    0 conv7x7, 1 bn1, 2 relu, 3 maxpool, 4 layer1, 5 layer2, 6 layer3, 7 layer4, 8 avgpool
    """
    assert isinstance(backbone, nn.Sequential), "backbone MUST be nn.Sequential"
    for p in backbone[stage_idx].parameters():
        p.requires_grad_(True)
    return f"backbone[{stage_idx}]"

In [ ]:
# === Loss of each target with mask and uncertainty weight ===
def masked_huber(pred, target, mask, delta=1.0):
    diff = pred - target
    absd = diff.abs()
    hub = torch.where(absd < delta, 0.5*diff*diff, delta*(absd - 0.5*delta))
    w = (mask > 0).float()
    return (hub * w).sum() / (w.sum() + 1e-8)

In [ ]:
def multitask_loss(outputs, y, m, model):
    # outputs: dict("ttc","dmin","tstar","dist") -> [B]
    # y: [B,4]  (y_ttc_inv, y_dmin_inv, y_tstar_norm, y_dist_inv)
    # m: [B,4]
    L_ttc   = masked_huber(outputs["ttc"],   y[:,0], m[:,0])
    L_dmin  = masked_huber(outputs["dmin"],  y[:,1], m[:,1])
    L_tstar = masked_huber(outputs["tstar"], y[:,2], m[:,2])
    L_dist  = masked_huber(outputs["dist"],  y[:,3], m[:,3])

    # Weighted sum by uncertainty weight (Use log_sigma which be learned automatically)
    def uw(name, L):
        log_sigma = model.log_sigma[name]
        return torch.exp(-2*log_sigma) * L + log_sigma
    loss = uw("ttc", L_ttc) + uw("dmin", L_dmin) + uw("tstar", L_tstar) + uw("dist", L_dist)

    logs = {"L_ttc": L_ttc.item(), "L_dmin": L_dmin.item(),
            "L_tstar": L_tstar.item(), "L_dist": L_dist.item()}
    return loss, logs

In [ ]:
# === 3) Optima/Scheduler ===
# Update only head/temporal when it's freezed → Filtering parameter
def optim_params(m):
    return [p for p in m.parameters() if p.requires_grad]

In [16]:
opt = torch.optim.AdamW(optim_params(model), lr=3e-4, weight_decay=1e-4)
sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=20)

In [ ]:
# === 4) Validation metric (MAE, physical units) ===
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve

EPS = 1e-3
H   = 4.0  # t* Horizon of used in normalization (index.csv 만들 때 썼던 거)

@torch.no_grad()
def evaluate_epoch(dataloader):
    model.eval()
    n = 0
    loss_sum = 0.0

    # Init MAE(expressed by physical units)
    mae_ttc = mae_dmin = mae_tstar = mae_dist = 0.0
    cnt_ttc = cnt_dmin = cnt_tstar = cnt_dist = 0

    # Buffer of sum for event-driven validation(whole validation)
    yT_gt=[]; yT_pr=[]; mT=[]
    yD_gt=[]; yD_pr=[]; mD=[]

    for xb, yb, mb in dataloader:
        xb, yb, mb = xb.to(device), yb.to(device), mb.to(device)
        out = model(xb)
        loss, logs = multitask_loss(out, yb, mb, model)
        loss_sum += loss.item() * xb.size(0)
        n += xb.size(0)

        # ---- Inverse transform ----
        # pred
        y_ttc_inv_pred   = out["ttc"]
        y_dmin_inv_pred  = out["dmin"]
        y_tstar_norm_pred= out["tstar"]
        y_dist_inv_pred  = out["dist"]
        # tgt
        y_ttc_inv_tgt    = yb[:,0]; y_dmin_inv_tgt = yb[:,1]
        y_tstar_norm_tgt = yb[:,2]; y_dist_inv_tgt = yb[:,3]

        # TTC [s]
        if mb[:,0].sum() > 0:
            ttc_pred = 1.0 / (y_ttc_inv_pred.clamp_min(1e-6) + EPS)
            ttc_tgt  = 1.0 / (y_ttc_inv_tgt.clamp_min(1e-6)  + EPS)
            diff = (ttc_pred - ttc_tgt).abs()
            mae_ttc += (diff * mb[:,0]).sum().item()
            cnt_ttc += mb[:,0].sum().item()

        # dmin [m]
        if mb[:,1].sum() > 0:
            dmin_pred = 1.0 / (y_dmin_inv_pred.clamp_min(1e-6) + EPS)
            dmin_tgt  = 1.0 / (y_dmin_inv_tgt.clamp_min(1e-6)  + EPS)
            diff = (dmin_pred - dmin_tgt).abs()
            mae_dmin += (diff * mb[:,1]).sum().item()
            cnt_dmin += mb[:,1].sum().item()

        # t* [s]
        if mb[:,2].sum() > 0:
            tstar_pred = y_tstar_norm_pred.clamp(0,1) * H
            tstar_tgt  = y_tstar_norm_tgt.clamp(0,1)  * H
            diff = (tstar_pred - tstar_tgt).abs()
            mae_tstar += (diff * mb[:,2]).sum().item()
            cnt_tstar += mb[:,2].sum().item()

        # dist [m]
        if mb[:,3].sum() > 0:
            dist_pred = 1.0 / (y_dist_inv_pred.clamp_min(1e-6) + EPS)
            dist_tgt  = 1.0 / (y_dist_inv_tgt.clamp_min(1e-6)  + EPS)
            diff = (dist_pred - dist_tgt).abs()
            mae_dist += (diff * mb[:,3]).sum().item()
            cnt_dist += mb[:,3].sum().item()

        # ---- Sum (Stay sorted) ----
        ttc_pr   = 1.0 / (out["ttc"].clamp_min(1e-6) + EPS)
        dmin_pr  = 1.0 / (out["dmin"].clamp_min(1e-6) + EPS)
        ttc_gt   = 1.0 / (yb[:,0].clamp_min(1e-6) + EPS)
        dmin_gt  = 1.0 / (yb[:,1].clamp_min(1e-6) + EPS)

        yT_gt.extend(ttc_gt.cpu().numpy());  yT_pr.extend(ttc_pr.cpu().numpy());  mT.extend(mb[:,0].cpu().numpy())
        yD_gt.extend(dmin_gt.cpu().numpy()); yD_pr.extend(dmin_pr.cpu().numpy()); mD.extend(mb[:,1].cpu().numpy())

    # Metric(MAE)
    logs = {
        "val_loss": loss_sum / max(1,n),
        "mae_ttc_s":   (mae_ttc  / max(1,cnt_ttc)),
        "mae_dmin_m":  (mae_dmin / max(1,cnt_dmin)),
        "mae_tstar_s": (mae_tstar/ max(1,cnt_tstar)),
        "mae_dist_m":  (mae_dist / max(1,cnt_dist)),
    }

    # Metric of event-driven classification(whole validation)
    yT_gt = np.array(yT_gt); yT_pr = np.array(yT_pr); mT = np.array(mT, dtype=bool)
    yD_gt = np.array(yD_gt); yD_pr = np.array(yD_pr); mD = np.array(mD, dtype=bool)
    common = mT & mD
    if common.sum() > 0:
        tgt_ttc  = yT_gt[common];  prd_ttc  = yT_pr[common]
        tgt_dmin = yD_gt[common];  prd_dmin = yD_pr[common]

        risk_gt = ((tgt_ttc < 2.5) | (tgt_dmin < 3.0)).astype(np.uint8)
        score   = (1.0/np.clip(prd_ttc,1e-3,None)) + (1.0/np.clip(prd_dmin,1e-3,None))

        roc = roc_auc_score(risk_gt, score)
        pr  = average_precision_score(risk_gt, score)
        prec, rec, thr = precision_recall_curve(risk_gt, score)
        f1 = 2*prec*rec/(prec+rec+1e-9)
        best_idx = int(np.argmax(f1))
        # Since return of precision_recall_curve, thr is shorter than prec/rec by 1
        best_thr = float(thr[max(best_idx-1, 0)]) if thr.size>0 else 0.5

        logs.update({
            "ev_n": int(common.sum()),
            "ev_roc_auc": float(roc),
            "ev_pr_auc":  float(pr),
            "ev_best_f1": float(f1[best_idx]),
            "ev_best_thr": best_thr,
            "ev_best_p":  float(prec[best_idx]),
            "ev_best_r":  float(rec[best_idx]),
        })
    else:
        logs.update({
            "ev_n": 0,
            "ev_roc_auc": np.nan,
            "ev_pr_auc":  np.nan,
            "ev_best_f1": np.nan,
            "ev_best_thr": np.nan,
            "ev_best_p":  np.nan,
            "ev_best_r":  np.nan,
        })

    return logs


In [ ]:
# === Train loop ===
EPOCHS = 25
BEST_KEY = "ev_pr_auc"
mode = 'max'
patience = 7
pat_cnt = 0
ckpt_path = "best_multitarget.pt"
best_val_ev_thr = None  # Save best F1 threshold when validate

def is_better(new, best, mode):
    eps = 1e-9
    return (new > best + eps) if mode == 'max' else (new < best - eps)

best_score = -float('inf') if mode == 'max' else float('inf')

for epoch in range(1, EPOCHS+1):
    t0 = time.time()
    model.train()

    # Unfreeze backbone + Reset optima
    if epoch == FREEZE_EPOCHS+1:
        unlocked = unfreeze_backbone_stage_by_index(model.backbone, stage_idx=7)
        model.backbone.apply(freeze_bn)
        print(f"[unfreeze] unlocked: {unlocked} (layer4)")
        bb_trainable_ids = {id(p) for p in model.backbone.parameters() if p.requires_grad}
        backbone_params = []
        head_params = []
        for _, p in model.named_parameters():
            if not p.requires_grad:
                continue
            (backbone_params if id(p) in bb_trainable_ids else head_params).append(p)
        opt = torch.optim.AdamW([
            {'params': head_params, 'lr': 3e-4, 'weight_decay': 1e-4},
            {'params': backbone_params, 'lr': 3e-5, 'weight_decay': 1e-5}
        ], betas=(0.9, 0.999))
        from torch.optim.lr_scheduler import SequentialLR, LinearLR, CosineAnnealingLR
        warmup = LinearLR(opt, start_factor=0.2, end_factor=1.0, total_iters=2)
        cosine = CosineAnnealingLR(opt, T_max=max(1, EPOCHS - FREEZE_EPOCHS - 2))
        sched = SequentialLR(opt, schedulers=[warmup, cosine], milestones=[2])
        print("[unfreeze] optimizer/scheduler reset.")

    # ---- train one epoch ----
    tr_loss = tr_Lttc = tr_Ldmin = tr_Ltstar = tr_Ldist = 0.0
    n = 0
    for xb, yb, mb in train_dl:
        xb, yb, mb = xb.to(device), yb.to(device), mb.to(device)
        opt.zero_grad(set_to_none=True)
        out = model(xb)
        loss, logs = multitask_loss(out, yb, mb, model)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        opt.step()

        bs = xb.size(0)
        tr_loss  += loss.item() * bs
        tr_Lttc  += logs["L_ttc"]  * bs
        tr_Ldmin += logs["L_dmin"] * bs
        tr_Ltstar+= logs["L_tstar"]* bs
        tr_Ldist += logs["L_dist"] * bs
        n += bs

    if sched is not None:
        sched.step()

    # ---- validation ----
    val_logs = evaluate_epoch(val_dl)

    # ---- logging ----
    tr_logs = {
        "train_loss": tr_loss/max(1,n),
        "L_ttc": tr_Lttc/max(1,n),
        "L_dmin": tr_Ldmin/max(1,n),
        "L_tstar": tr_Ltstar/max(1,n),
        "L_dist": tr_Ldist/max(1,n),
    }
    elapsed = time.time() - t0
    print(f"[{epoch:02d}] "
          f"train {tr_logs['train_loss']:.4f} "
          f"(ttc {tr_logs['L_ttc']:.4f} dmin {tr_logs['L_dmin']:.4f} "
          f"t* {tr_logs['L_tstar']:.4f} dist {tr_logs['L_dist']:.4f})  |  "
          f"val {val_logs['val_loss']:.4f} "
          f"[MAE: ttc {val_logs['mae_ttc_s']:.3f}s, "
          f"dmin {val_logs['mae_dmin_m']:.3f}m, "
          f"t* {val_logs['mae_tstar_s']:.3f}s, "
          f"dist {val_logs['mae_dist_m']:.3f}m]  "
          f"| [EVENT: N={val_logs['ev_n']} "
          f"ROC-AUC {val_logs['ev_roc_auc']:.3f} "
          f"PR-AUC {val_logs['ev_pr_auc']:.3f} "
          f"BestF1 {val_logs['ev_best_f1']:.3f} @thr≈{val_logs['ev_best_thr']:.3f} "
          f"(P {val_logs['ev_best_p']:.3f}, R {val_logs['ev_best_r']:.3f})] "
          f"({elapsed:.1f}s)")

    # ---- checkpoint & early stopping ----
    score = val_logs[BEST_KEY]
    if is_better(score, best_score, mode):
        best_score = score
        pat_cnt = 0
        best_val_ev_thr = val_logs.get('ev_best_thr', None)
        torch.save({
            "epoch": epoch,
            "model_state": model.state_dict(),
            "opt_state": opt.state_dict(),
            "best_score": best_score,
            'best_val_ev_thr': best_val_ev_thr,
            "cfg": {"T_LEN": T_LEN, "IMG_SIZE": IMG_SIZE}
        }, ckpt_path)
        print(f"  ↳ saved best to {ckpt_path} (ev_pr_auc={best_score:.4f})")
    else:
        pat_cnt += 1
        if pat_cnt >= patience:
            print("Early stopping triggered.")
            break

print("done. best ev_pr_auc:", best_score)

[01] train -0.0638 (ttc 0.0269 dmin 0.0081 t* 0.0676 dist 0.0020)  |  val -0.1490 [MAE: ttc 2.221s, dmin 5.095m, t* 1.343s, dist 8.021m]  | [EVENT: N=428 ROC-AUC 0.617 PR-AUC 0.534 BestF1 0.596 @thr≈0.374 (P 0.545, R 0.657)] (152.8s)
  ↳ saved best to best_multitarget.pt (ev_pr_auc=0.5336)
[02] train -0.4185 (ttc 0.0204 dmin 0.0069 t* 0.0555 dist 0.0018)  |  val -0.4642 [MAE: ttc 1.838s, dmin 4.585m, t* 1.415s, dist 7.955m]  | [EVENT: N=428 ROC-AUC 0.802 PR-AUC 0.734 BestF1 0.699 @thr≈0.389 (P 0.565, R 0.916)] (153.6s)
  ↳ saved best to best_multitarget.pt (ev_pr_auc=0.7343)
[03] train -0.7512 (ttc 0.0167 dmin 0.0062 t* 0.0517 dist 0.0017)  |  val -0.7691 [MAE: ttc 1.879s, dmin 4.618m, t* 1.357s, dist 7.486m]  | [EVENT: N=428 ROC-AUC 0.755 PR-AUC 0.701 BestF1 0.716 @thr≈0.381 (P 0.677, R 0.759)] (153.7s)
[04] train -1.0674 (ttc 0.0160 dmin 0.0059 t* 0.0478 dist 0.0017)  |  val -1.0304 [MAE: ttc 1.996s, dmin 4.689m, t* 1.335s, dist 8.130m]  | [EVENT: N=428 ROC-AUC 0.774 PR-AUC 0.689 Bes

/home/bocchi/workspace/capstone/code/.venv/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:209: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


[07] train -1.6949 (ttc 0.0096 dmin 0.0044 t* 0.0399 dist 0.0016)  |  val -1.5340 [MAE: ttc 1.820s, dmin 4.695m, t* 1.358s, dist 8.125m]  | [EVENT: N=428 ROC-AUC 0.794 PR-AUC 0.714 BestF1 0.688 @thr≈0.445 (P 0.561, R 0.892)] (151.9s)
[08] train -1.9612 (ttc 0.0098 dmin 0.0045 t* 0.0388 dist 0.0016)  |  val -1.8479 [MAE: ttc 1.735s, dmin 4.287m, t* 1.307s, dist 7.359m]  | [EVENT: N=428 ROC-AUC 0.850 PR-AUC 0.792 BestF1 0.754 @thr≈0.350 (P 0.733, R 0.777)] (158.0s)
  ↳ saved best to best_multitarget.pt (ev_pr_auc=0.7917)
[09] train -2.3048 (ttc 0.0107 dmin 0.0042 t* 0.0328 dist 0.0016)  |  val -2.1417 [MAE: ttc 1.928s, dmin 4.626m, t* 1.280s, dist 7.775m]  | [EVENT: N=428 ROC-AUC 0.748 PR-AUC 0.703 BestF1 0.656 @thr≈0.527 (P 0.581, R 0.753)] (152.4s)
[10] train -2.6421 (ttc 0.0075 dmin 0.0035 t* 0.0290 dist 0.0016)  |  val -2.4231 [MAE: ttc 1.765s, dmin 4.440m, t* 1.245s, dist 7.579m]  | [EVENT: N=428 ROC-AUC 0.802 PR-AUC 0.771 BestF1 0.671 @thr≈0.666 (P 0.812, R 0.572)] (157.2s)
[11] tr

In [19]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, precision_recall_curve, confusion_matrix, ConfusionMatrixDisplay

def evaluate_dataset_event(dataloader, fixed_threshold=None, save_prefix="test"):
    model.eval()
    yT_gt=[]; yT_pr=[]; mT=[]
    yD_gt=[]; yD_pr=[]; mD=[]
    with torch.no_grad():
        for xb, yb, mb in dataloader:
            xb, yb, mb = xb.to(device), yb.to(device), mb.to(device)
            out = model(xb)
            # 역변환
            ttc_pr   = 1.0 / (out["ttc"].clamp_min(1e-4) + EPS)
            dmin_pr  = 1.0 / (out["dmin"].clamp_min(1e-4) + EPS)
            ttc_gt   = 1.0 / (yb[:,0].clamp_min(1e-4) + EPS)
            dmin_gt  = 1.0 / (yb[:,1].clamp_min(1e-4) + EPS)
            # 누적
            yT_gt.extend(ttc_gt.cpu().numpy());  yT_pr.extend(ttc_pr.cpu().numpy());  mT.extend(mb[:,0].cpu().numpy())
            yD_gt.extend(dmin_gt.cpu().numpy()); yD_pr.extend(dmin_pr.cpu().numpy()); mD.extend(mb[:,1].cpu().numpy())

    yT_gt = np.array(yT_gt); yT_pr = np.array(yT_pr); mT = np.array(mT, dtype=bool)
    yD_gt = np.array(yD_gt); yD_pr = np.array(yD_pr); mD = np.array(mD, dtype=bool)
    common = mT & mD
    assert common.sum() > 0, "No common valid TTC & dmin samples in test set."

    tgt_ttc  = yT_gt[common];  prd_ttc  = yT_pr[common]
    tgt_dmin = yD_gt[common];  prd_dmin = yD_pr[common]

    risk_gt = ((tgt_ttc < 2.5) | (tgt_dmin < 3.0)).astype(np.uint8)
    score   = (1.0/np.clip(prd_ttc,1e-4,None)) + (1.0/np.clip(prd_dmin,1e-4,None))

    # 곡선/면적
    fpr, tpr, roc_thr = roc_curve(risk_gt, score)
    roc_auc = roc_auc_score(risk_gt, score)
    prec, rec, pr_thr = precision_recall_curve(risk_gt, score)
    pr_auc  = average_precision_score(risk_gt, score)

    # 테스트 자체에서의 best-F1 (참고용)
    f1 = 2*prec*rec/(prec+rec+1e-9)
    best_idx = int(np.argmax(f1))
    best_f1  = float(f1[best_idx])
    best_thr_test = float(pr_thr[max(best_idx-1,0)]) if pr_thr.size>0 else 0.5

    # 최종 리포트용 Confusion Matrix 임계값: 고정 임계값 사용(검증에서 저장한 값)
    thr_used = float(fixed_threshold) if fixed_threshold is not None else best_thr_test
    y_pred = (score >= thr_used).astype(np.uint8)
    cm = confusion_matrix(risk_gt, y_pred, labels=[0,1])
    tn, fp, fn, tp = cm.ravel()
    P = tp / max(tp+fp, 1e-9)
    R = tp / max(tp+fn, 1e-9)
    F1_at_thr = 2*P*R / max(P+R, 1e-9)

    # --- 그림 저장 ---
    # 1) ROC
    plt.figure()
    plt.plot(fpr, tpr, label=f"ROC (AUC={roc_auc:.3f})")
    plt.plot([0,1],[0,1],'--')
    plt.xlabel("FPR")
    plt.ylabel("TPR")
    plt.legend(loc="lower right")
    plt.title("ROC Curve")
    plt.tight_layout()
    plt.savefig(f"{save_prefix}_roc.png", dpi=200)
    plt.close()

    # 2) PR
    plt.figure()
    plt.plot(rec, prec, label=f"PR (AP={pr_auc:.3f})")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.legend(loc="lower left")
    plt.title("Precision-Recall Curve")
    plt.tight_layout()
    plt.savefig(f"{save_prefix}_pr.png", dpi=200)
    plt.close()

    # 3) 점수 히스토그램
    plt.figure()
    plt.hist(score[risk_gt==0], bins=40, alpha=0.7, label="safe")
    plt.hist(score[risk_gt==1], bins=40, alpha=0.7, label="risk")
    plt.axvline(thr_used, linestyle="--", label=f"thr={thr_used:.3f}")
    plt.xlabel("risk score")
    plt.ylabel("count")
    plt.legend()
    plt.title("Risk Score Distribution")
    plt.tight_layout()
    plt.savefig(f"{save_prefix}_score_hist.png", dpi=200)
    plt.close()

    # 4) Confusion Matrix (thr_used)
    disp = ConfusionMatrixDisplay(cm, display_labels=["safe(0)","risk(1)"])
    fig, ax = plt.subplots()
    disp.plot(ax=ax, values_format="d", colorbar=False)
    ax.set_title(f"Confusion Matrix @thr={thr_used:.3f}")
    plt.tight_layout()
    plt.savefig(f"{save_prefix}_cm.png", dpi=200)
    plt.close()

    print("score stats:",
      float(score.min()), float(score.max()),
      "thr_used:", float(thr_used),
      "pos_rate:", float((score >= thr_used).mean()))

    return {
        "N": int(common.sum()),
        "roc_auc": float(roc_auc),
        "pr_auc": float(pr_auc),
        "best_f1_test": best_f1,
        "best_thr_test": best_thr_test,
        "thr_used": thr_used,
        "cm": {"tn": int(tn), "fp": int(fp), "fn": int(fn), "tp": int(tp)},
        "P_at_thr": float(P),
        "R_at_thr": float(R),
        "F1_at_thr": float(F1_at_thr),
        "figs": {
            "roc": f"{save_prefix}_roc.png",
            "pr":  f"{save_prefix}_pr.png",
            "hist":f"{save_prefix}_score_hist.png",
            "cm":  f"{save_prefix}_cm.png",
        }
    }


In [ ]:
ckpt = torch.load(ckpt_path, map_location="cpu")
model.load_state_dict(ckpt["model_state"])
model.to(device)
best_val_ev_thr = ckpt.get("best_val_ev_thr", None)
print("best_val_ev_thr from validation:", best_val_ev_thr)

test_report = evaluate_dataset_event(test_dl, fixed_threshold=best_val_ev_thr, save_prefix="test")
print(test_report)
# => test_roc.png, test_pr.png, test_score_hist.png, test_cm.png

best_val_ev_thr from validation: 0.5223032236099243
score stats: 0.22154535353183746 1.3180476427078247 thr_used: 0.5223032236099243 pos_rate: 0.46238938053097345
{'N': 452, 'roc_auc': 0.8167225511878434, 'pr_auc': 0.7143961705534915, 'best_f1_test': 0.6411149820821911, 'best_thr_test': 0.5537711381912231, 'thr_used': 0.5223032236099243, 'cm': {'tn': 211, 'fp': 110, 'fn': 32, 'tp': 99}, 'P_at_thr': 0.47368421052631576, 'R_at_thr': 0.7557251908396947, 'F1_at_thr': 0.5823529411764706, 'figs': {'roc': 'test_roc.png', 'pr': 'test_pr.png', 'hist': 'test_score_hist.png', 'cm': 'test_cm.png'}}


---
## For experiment

In [22]:
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score, roc_auc_score, average_precision_score, precision_recall_curve

model.eval()
# 원본(정렬 유지) 리스트들
yT_gt=[]; yT_pr=[]; mT=[]
yD_gt=[]; yD_pr=[]; mD=[]
yS_gt=[]; yS_pr=[]; mS=[]
yC_gt=[]; yC_pr=[]; mC=[]

with torch.no_grad():
    for xb, yb, mb in val_dl:
        xb, yb, mb = xb.to(device), yb.to(device), mb.to(device)
        out = model(xb)

        # 역변환 (예측)
        ttc_pr   = 1.0 / (out["ttc"].clamp_min(1e-6) + EPS)
        dmin_pr  = 1.0 / (out["dmin"].clamp_min(1e-6) + EPS)
        tstar_pr = out["tstar"].clamp(0,1) * H
        dist_pr  = 1.0 / (out["dist"].clamp_min(1e-6) + EPS)

        # 역변환 (GT)
        ttc_gt   = 1.0 / (yb[:,0].clamp_min(1e-6) + EPS)
        dmin_gt  = 1.0 / (yb[:,1].clamp_min(1e-6) + EPS)
        tstar_gt = yb[:,2].clamp(0,1) * H
        dist_gt  = 1.0 / (yb[:,3].clamp_min(1e-6) + EPS)

        # 그대로(정렬 유지) 누적 + 마스크도 함께 저장
        yT_gt.extend(ttc_gt.cpu().numpy());   yT_pr.extend(ttc_pr.cpu().numpy());   mT.extend(mb[:,0].cpu().numpy())
        yD_gt.extend(dmin_gt.cpu().numpy());  yD_pr.extend(dmin_pr.cpu().numpy());  mD.extend(mb[:,1].cpu().numpy())
        yS_gt.extend(tstar_gt.cpu().numpy()); yS_pr.extend(tstar_pr.cpu().numpy()); mS.extend(mb[:,2].cpu().numpy())
        yC_gt.extend(dist_gt.cpu().numpy());  yC_pr.extend(dist_pr.cpu().numpy());  mC.extend(mb[:,3].cpu().numpy())

# 넘파이 변환
yT_gt=np.array(yT_gt); yT_pr=np.array(yT_pr); mT=np.array(mT).astype(bool)
yD_gt=np.array(yD_gt); yD_pr=np.array(yD_pr); mD=np.array(mD).astype(bool)
yS_gt=np.array(yS_gt); yS_pr=np.array(yS_pr); mS=np.array(mS).astype(bool)
yC_gt=np.array(yC_gt); yC_pr=np.array(yC_pr); mC=np.array(mC).astype(bool)

# --- 1) Per-task MAE/R2 (각 타깃 마스크로 별도 집계) ---
def report(name, gt, pr, mask, unit):
    if mask.sum()==0:
        print(f"{name}: (no valid samples)")
        return
    print(f"{name}: MAE={mean_absolute_error(gt[mask], pr[mask]):.3f}{unit}  "
          f"R2={r2_score(gt[mask], pr[mask]):.3f}")

print("[VAL MAE/R2]")
report("TTC",  yT_gt, yT_pr, mT, "s")
report("dmin", yD_gt, yD_pr, mD, "m")
report("t*",   yS_gt, yS_pr, mS, "s")
report("dist", yC_gt, yC_pr, mC, "m")

# --- 2) 이벤트 기반 평가: 공통 마스크 사용 ---
# 위험 = (TTC < 2.5) OR (dmin < 3.0)
common = mT & mD           # 두 타깃이 동시에 유효한 샘플만
if common.sum() == 0:
    print("[VAL EVENT] skip (no common valid TTC & dmin samples)")
else:
    tgt_ttc  = yT_gt[common];  prd_ttc  = yT_pr[common]
    tgt_dmin = yD_gt[common];  prd_dmin = yD_pr[common]

    risk_gt = ((tgt_ttc < 2.5) | (tgt_dmin < 3.0)).astype(np.uint8)
    # 위험 스코어 (클수록 위험): 1/TTC + 1/dmin
    score = (1.0/np.clip(prd_ttc,1e-3,None)) + (1.0/np.clip(prd_dmin,1e-3,None))
    score = (score - score.min()) / (score.max() - score.min() + 1e-9)

    from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve
    auc = roc_auc_score(risk_gt, score)
    ap  = average_precision_score(risk_gt, score)
    prec, rec, thr = precision_recall_curve(risk_gt, score)
    f1 = 2*prec*rec/(prec+rec+1e-9)
    best = np.argmax(f1)

    print(f"[VAL EVENT] N={common.sum()}  ROC-AUC={auc:.3f}  PR-AUC={ap:.3f}  "
          f"Best F1={f1[best]:.3f} at thr≈{thr[max(best-1,0)]:.3f} "
          f"(P={prec[best]:.3f}, R={rec[best]:{'.3f'}})")


[VAL MAE/R2]
TTC: MAE=1.896s  R2=-0.013
dmin: MAE=4.033m  R2=-0.208
t*: MAE=1.248s  R2=0.054
dist: MAE=3.819m  R2=0.294
[VAL EVENT] N=232  ROC-AUC=0.902  PR-AUC=0.856  Best F1=0.854 at thr≈0.346 (P=0.880, R=0.830)


In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score, roc_auc_score, average_precision_score, precision_recall_curve

# 위에서 만든 yT_gt/yT_pr/... mT/mD 그대로 사용
common = mT & mD
tgt_ttc  = yT_gt[common];  prd_ttc  = yT_pr[common]
tgt_dmin = yD_gt[common];  prd_dmin = yD_pr[common]

# --- 회귀 베이스라인: 각 타깃의 검증 평균치 예측 ---
mean_ttc  = tgt_ttc.mean()
mean_dmin = tgt_dmin.mean()
bl_ttc  = np.full_like(tgt_ttc,  mean_ttc)
bl_dmin = np.full_like(tgt_dmin, mean_dmin)

print("[NAIVE REG BASELINE]")
print(f"TTC:  MAE={mean_absolute_error(tgt_ttc, bl_ttc):.3f}s  R2={r2_score(tgt_ttc, bl_ttc):.3f}")
print(f"dmin: MAE={mean_absolute_error(tgt_dmin, bl_dmin):.3f}m  R2={r2_score(tgt_dmin, bl_dmin):.3f}")

# --- 이벤트 베이스라인: 단일 상수 스코어(무의미) 대신 안전하게 rule 기반 ---
# 위험 GT
risk_gt = ((tgt_ttc < 2.5) | (tgt_dmin < 3.0)).astype(np.uint8)
# 베이스라인 스코어: 예측 평균치로부터 1/ttc + 1/dmin 계산
score_bl = (1.0/np.clip(bl_ttc,1e-3,None)) + (1.0/np.clip(bl_dmin,1e-3,None))
score_bl = (score_bl - score_bl.min()) / (score_bl.max() - score_bl.min() + 1e-9)

auc_bl = roc_auc_score(risk_gt, score_bl)
ap_bl  = average_precision_score(risk_gt, score_bl)
print(f"[NAIVE EVENT BASELINE] ROC-AUC={auc_bl:.3f}  PR-AUC={ap_bl:.3f}")


[NAIVE REG BASELINE]
TTC:  MAE=2.274s  R2=0.000
dmin: MAE=4.815m  R2=0.000
[NAIVE EVENT BASELINE] ROC-AUC=0.500  PR-AUC=0.457


In [24]:
import pandas as pd
tr=pd.read_csv("index_train.csv"); va=pd.read_csv("index_val.csv")
te=pd.read_csv("index_test.csv")
assert set(tr.clip_id).isdisjoint(va.clip_id) and set(tr.clip_id).isdisjoint(te.clip_id) and set(va.clip_id).isdisjoint(te.clip_id)

In [26]:
import pandas as pd
df=pd.read_csv("index_train.csv")
print("mask_ttc=1 ratio:", df["mask_ttc"].mean())
print("mask_dmin=1 ratio:", df["mask_dmin"].mean())
df=pd.read_csv("index_val.csv")
print("mask_ttc=1 ratio:", df["mask_ttc"].mean())
print("mask_dmin=1 ratio:", df["mask_dmin"].mean())


mask_ttc=1 ratio: 0.8722689075630252
mask_dmin=1 ratio: 0.980672268907563
mask_ttc=1 ratio: 0.8560885608856088
mask_dmin=1 ratio: 0.981549815498155


In [27]:
# (평가 때 썼던 common 마스크 기준) 위험 비율 확인
pos_rate = ((tgt_ttc < 2.5) | (tgt_dmin < 3.0)).mean()
print("VAL positive rate:", pos_rate)

VAL positive rate: 0.45689655172413796


In [28]:
from collections import Counter
from itertools import chain
paths = list(chain.from_iterable(df[[f"f{k}" for k in range(16)]].values.tolist()))
dup = [p for p,c in Counter(paths).items() if c>1]
print("dup frames:", len(dup))

dup frames: 1116


In [29]:
import os
paths = list(chain.from_iterable(df[[f"f{k}" for k in range(16)]].values.tolist()))
unique_paths = set(paths)
print("전체 프레임 수:", len(paths))
print("고유 프레임 수:", len(unique_paths))
print("중복 비율:", 1 - len(unique_paths)/len(paths))


전체 프레임 수: 4336
고유 프레임 수: 1180
중복 비율: 0.7278597785977861
